In [1]:
import pandas as pd 
import numpy as np 
from scipy.stats import gamma, dirichlet, beta, nbinom


In [2]:
#Load dataset
filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final.csv"
#filename2 = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\tcga_rna.txt"
#filename1 = "C:\\Users\\raoki\\Documents\\GitHub\\project_spring2019\\Data\\tcga_cli.txt"
#filename2 = "C:\\Users\\raoki\\Documents\\GitHub\\project_spring2019\\Data\\tcga_rna.txt"
data = pd.read_csv(filename1, sep=';')
#datar = pd.read_csv(filename2, sep=';')

In [3]:
#Checking the size of the datasets
print('clinical data: ',datac.shape, '\nRNA data:', datar.shape)

#Creating a variable indicator with the prediction value in the 0/1 format
datac['y'] = datac['new_tumor_event_dx_indicator']
datac['y'] = datac['y'].replace(['NO','YES'],[0,1])
datac['gender'] = datac['gender'].replace(['MALE','FEMALE'],[0,1])

#Checking if the values are ok
print(datac['new_tumor_event_dx_indicator'].value_counts(),'\n', datac['y'].value_counts())

clinical data:  (2854, 9) 
RNA data: (2854, 20533)
NO     1815
YES    1039
Name: new_tumor_event_dx_indicator, dtype: int64 
 0    1815
1    1039
Name: y, dtype: int64


In [4]:
#selecting important clinical features and creating dummy/binary variables for future use
datac_s = datac[['patients','gender','abr','y']]
datac_s = pd.get_dummies(datac_s,columns =['abr'])
datac_s.head()

,patients,gender,y,abr_ACC,abr_BLCA,abr_CHOL,abr_ESCA,abr_HNSC,abr_LGG,abr_LIHC,abr_LUSC,abr_MESO,abr_PAAD,abr_PRAD,abr_SARC,abr_SKCM,abr_TGCT,abr_UCS
36,TCGA-21-A5DI,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
73,TCGA-22-A5C4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
74,TCGA-2A-A8VL,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
75,TCGA-2A-A8VO,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
76,TCGA-2A-A8VT,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [5]:
#raw counts weren't integer, so I will use a round function to round it. 
datar = datar.drop(columns = 'patients2')
print(datar.head())
for i in np.arange(1,datar.shape[1]):
    datar.iloc[:,i] = round(datar.iloc[:,i])
    
print(datar.head())

        patients  g100130426  g100133144  g100134869  g10357  g10431  g136542  \
36  TCGA-21-A5DI         0.0       25.92       27.08  154.92    1379        0   
73  TCGA-22-A5C4         0.0      101.50       43.50  269.56    2246        0   
74  TCGA-2A-A8VL         0.0       40.30       33.70  157.13    2546        0   
75  TCGA-2A-A8VO         0.0       29.07       20.93   93.40    1300        0   
76  TCGA-2A-A8VT         0.0      132.97      161.03  256.83    1585        0   

    g155060  g26823  g280660  ...  ZXDA  ZXDB  ZXDC  ZYG11A  ZYG11B      ZYX  \
36      584       0        0  ...    57   422  2913      96   689.0  12608.0   
73      667      25        0  ...   120  1082  3180     712  2658.0   9258.0   
74      612       0        0  ...    93   552  2139      74  1344.0   4474.0   
75      569       0        0  ...    63   577  1366     252  1138.0   4081.0   
76     1985       3        0  ...   308  1629  4699     121  3239.0   4174.0   

    ZZEF1  ZZZ3  psiTPTE22  tAKR

In [6]:
#Merging two datasets. First columns have the clinical information.
print('shapes before: ',datac_s.shape, datar.shape)
data = pd.merge(datac_s,datar,on='patients')
print('shape after', data.shape)

shapes before:  (2854, 18) (2854, 20532)


KeyboardInterrupt: 

In [33]:
#Class to keep the parameters. WORK IN PROGRESS
class parameters:
    def __init__(self, latent_v,latent_sk,latent_pj,latent_phi ,latent_tht, prediction):
        self.ln = latent_v #array with parameters that are only one number 
        self.la_sk = latent_sk #array 
        self.la_pj = latent_pj #array
        self.lm_phi = latent_phi #matrix 
        self.lm_tht = latent_tht #matrix        
        self.p = prediction #array [intercept, gender, 15 cancer types, k genes]
        

In [12]:
#Declaring my priori values: param.ln = [0-c0,1-gamma0,2-eta,3-cj], param.la_sk, param.la_pj, param.lm_phi, param.lm_tht and param.p = [] 
#j=sample of patients 
#v=genes
#k=latent
#UPDATE PARAM 
def priori_l(param):
    k = len(param.la_sk)
    j = len(param.la_pj)
    #Prioris on the K plate
    s0 = 1 #define better numbers in the future
    t0 = 1 #define better numbers in the future 
    c0 = gamma.pdf(x=param.ln[0],a = s0,scale = 1/t0)
    a0 = 1 #define better numbers in the futureT
    b0 = 1 #define better numbers in the future
    gamma0 = gamma.pdf(x=param.ln[1],a = a0,scale = 1/b0)
    sk = gamma.pdf(x=param.la_sk,a=gamma0/k,scale=1/c0)#array
    s0 = 1
    w0 = 1
    eta = gamma.pdf(x=param.ln[2], a = s0, scale = w0)
    phivk = dirichlet.pdf(param.lm_phi, np.repeat(eta,j)) #MATRIX
    priorik = np.log(c0)+np.log(gamma0)+np.log(sk).sum()+np.log(eta)+np.log(phivk).sum()
    #priories on the J plate
    a0 = 1 #define better numbers in the future
    b0 = 1 #define better numbers in the future
    pj = beta(x=param.la_pj,a=a0,b=b0) #same numbers? #array
    e0 = 1
    f0 = 1
    cj = gamma.pdf(param.ln[3], a = e0, scale = f0)
    priorij = np.log(pj).sum()+np.log(cj) #array    
    #prioris on K and J plate
    thetakj = gamma.pdf(x=param.lm_tht,a=sk,scale = 1/cj) #MATRIX
    priorikj = np.log(thetakj).sum()
    
    return (priorik+priorij+priorikj)
    
def priori_p(param):
    #add prioris of logistic regression 
    #features are the theta_kj parameters and clinical info
    #one parameter for each latent variable + one parameter for each clinical features
    k = len(param.la_sk)
    mean = len(param.p)
    sd = 5
    mean.expend(1,np.repeat(1/15, 15))
    sd.expend(0.05,np.repeat(0.05/15,15))
    mean.expend(np.repeat(1,k))
    sd.expend(np.repeat(0.05,k))
    lr = normal.pdf(param.p,loc = mean, scale = sd) #loc/mean, scale/sd

In [ ]:
#Declare all the likelihood distributions in two separete functions
#_F represents the factor model likelihood and _P the prediction model likelihood

#Gene expression follows a gamma distribution 
def likelihood_F(param,data):
    phi_theta = np.matmul(param.lm_phi, param.lm_tht) #phi(jk) x theta(kv) must be matrix , final dim is j patients x v genes 
    nvj = nbinom.pmf(,n = phi_theta,p= pj) #matrix and array?
    return np.log(nvj).sum()

#Prediction is a logistic regression 
def likelihood_P(param, data):
    xw = param.p[0]+param.p[1]*data['gender']+
    
    logistic.pdf.equation
    
    
    
    return np.log(l1).sum()

In [ ]:
#combine the prioris and the likelihood 
def posterior(param,data,k):
    j = data.shape[0]
    return (likelihood_F(param,data)+likelihood_P(param, data)+priori_l(param,k,j)+priori_p(param,k,j)

In [ ]:
#These two functions propose new parameters candidates 
def proposal_f(current):
    #every parameter is a normal distrbituion and need to define sd
    new.f = np.random.normal(current.f)#define better numbers in the future
    return new

def proposal_p(current):
    #every parameter is a normal distrbituion and need to define sd
     new.p = np.random.normal(current.p)#define better numbers in the future
    return new

In [ ]:
#Itaration part. It works with two separate but sequential chains. 
def MCMC(startvalue, iterations, data,k):
    chain_f = np.zeros((iterations+1,len(startvalue.classinfo)))
    chain_p = np.zeros((iterations+1,len(startvalue.classinfo)))
    chain1[0]=proposalvalues1(startvalue1)
    chain2[0]=proposalvalues2(startvalue2)
    for i in np.arange(iterations):
        param_new.f = proposal_f(chain_f[i,])
        param_curr.f = chain_f[i-1,]
        param_new.p = param_curr.p = chain_p[i-1,]  
        prob_f = np.exp(posterior(param_new,data,k)-posterior(param_curr,data,k))
        if np.random.uniform(0,1,1)<prob_f:
            chain_f[i+1,] = param_new.f
        else:
            chain_f[i+1,] = chain_f[i,]
            
        #repeat process to prediction    

In [45]:
#dirichlet.pdf([0.5],shape= [0.5])
#quantiles = np.array([0.2, 0.2, 0.6])
#alpha = np.array([0.4, 5, 15]) 
#dirichlet.pdf(quantiles, alpha)
#gamma.pdf([0.5,0.75], a = [1,5], scale =1)
#nbinom.pmf(2,n = 2,p=0.5)
a = [1,2,3]
b = [4,5]
a.extend(b)
a

[1, 2, 3, 4, 5]